In [1]:
%run dataFormating.ipynb

rmdf152 read_csv success


../Utilities/Preparation.ipynb:2: ParserWarning: Both a converter and dtype were specified for column customData.localplayerguid - only the converter will be used
  "cells": [


gformEN read_csv success
gformFR read_csv success
temporalities set


FloatProgress(value=0.0, max=412.0)

dataFormating.ipynb:9: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  "outputs": [


# What subsets of scientific questions tend to be answered correctly by the same subjects?

## Mining

In [2]:
from orangecontrib.associate.fpgrowth import *  
import pandas as pd
from numpy import *

In [3]:
questions = correctedScientific.columns
correctedScientificText = [[] for _ in range(correctedScientific.shape[0])]
for q in questions:
    for r in range(correctedScientific.shape[0]):
        if correctedScientific.loc[r, q]:
            correctedScientificText[r].append(q)
#correctedScientificText

In [4]:
# Get frequent itemsets with support > 25%
# run time < 1 min
itemsets = frequent_itemsets(correctedScientificText, math.floor(len(correctedScientificText) * 0.25))
#dict(itemsets)

In [5]:
# Generate rules according to confidence, confidence > 85 %
# run time < 5 min
rules = association_rules(dict(itemsets), 0.85)
#list(rules)

In [6]:
# Transform rules generator into a Dataframe
rulesDataframe = pd.DataFrame([(ant, cons, supp, conf) for ant, cons, supp, conf in rules])
rulesDataframe.rename(columns = {0:"antecedants", 1:"consequents", 2:"support", 3:"confidence"}, inplace=True)
rulesDataframe.head()

,antecedants,consequents,support,confidence
0,"(Q20, Q21, Q26, Q19, Q1)",(Q27),134,0.943662
1,"(Q21, Q26, Q19, Q1)","(Q20, Q27)",134,0.905405
2,"(Q21, Q26, Q1)","(Q20, Q27, Q19)",134,0.893333
3,"(Q20, Q1, Q26, Q19)","(Q27, Q21)",134,0.924138
4,"(Q1, Q19, Q26)","(Q20, Q27, Q21)",134,0.858974


In [7]:
# Save the mined rules to file
rulesDataframe.to_csv("results/associationRulesMiningSupport25percentsConfidence85percents.csv")

## Search for interesting rules
Interesting rules are more likely to be the ones with highest confidence, the highest lift or with a bigger consequent set. Pairs can also be especially interesting

In [8]:
# Sort rules by confidence
confidenceSortedRules = rulesDataframe.sort_values(by = ["confidence", "support"], ascending=[False, False])
confidenceSortedRules.head(50)

,antecedants,consequents,support,confidence
418,"(Q27, Q21, Q19)",(Q26),153,1.0
717,"(Q20, Q27, Q21)",(Q26),147,1.0
248,"(Q20, Q27, Q21, Q19)",(Q26),146,1.0
699,"(Q20, Q21, Q1)",(Q19),145,1.0
290,"(Q20, Q21, Q26, Q1)",(Q19),142,1.0
767,"(Q27, Q21, Q1)",(Q26),141,1.0
267,"(Q27, Q21, Q1, Q19)",(Q26),139,1.0
933,"(Q21, Q22)",(Q19),137,1.0
21,"(Q20, Q27, Q21, Q1, Q19)",(Q26),134,1.0
32,"(Q20, Q27, Q21, Q1)","(Q26, Q19)",134,1.0


In [9]:
# Sort rules by size of consequent set
rulesDataframe["consequentSize"] = rulesDataframe["consequents"].apply(lambda x: len(x))
consequentSortedRules = rulesDataframe.sort_values(by = ["consequentSize", "confidence", "support"], ascending=[False, False, False])
consequentSortedRules.head(50)

,antecedants,consequents,support,confidence,consequentSize
52,"(Q20, Q15)","(Q27, Q21, Q26, Q19)",124,0.953846,4
49,"(Q15, Q21)","(Q20, Q27, Q26, Q19)",124,0.918519,4
11,"(Q20, Q1)","(Q27, Q21, Q26, Q19)",134,0.881579,4
30,"(Q20, Q27)","(Q21, Q26, Q19, Q1)",134,0.875817,4
51,"(Q15, Q19)","(Q20, Q27, Q21, Q26)",124,0.867133,4
9,"(Q21, Q1)","(Q20, Q27, Q26, Q19)",134,0.864516,4
191,(Q22),"(Q20, Q21, Q26, Q19)",126,0.863014,4
342,(Q22),"(Q27, Q21, Q26, Q19)",125,0.856164,4
242,(Q20),"(Q27, Q21, Q26, Q19)",146,0.853801,4
25,"(Q27, Q21)","(Q20, Q1, Q26, Q19)",134,0.853503,4


In [10]:
# Select only pairs (rules with antecedent and consequent of size one)
# Sort pairs according to confidence
rulesDataframe["fusedRule"] = rulesDataframe[["antecedants", "consequents"]].apply(lambda x: frozenset().union(*x), axis=1)
rulesDataframe["ruleSize"] = rulesDataframe["fusedRule"].apply(lambda x: len(x))
pairRules = rulesDataframe.sort_values(by=["ruleSize", "confidence", "support"], ascending=[True, False, False])
pairRules.head(30)

,antecedants,consequents,support,confidence,consequentSize,fusedRule,ruleSize
1091,(Q20),(Q19),168,0.982456,1,"(Q20, Q19)",2
1043,(Q22),(Q19),142,0.972603,1,"(Q19, Q22)",2
1082,(Q21),(Q19),169,0.971264,1,"(Q21, Q19)",2
1062,(Q17),(Q15),134,0.971014,1,"(Q15, Q17)",2
1089,(Q24),(Q26),132,0.963504,1,"(Q24, Q26)",2
1092,(Q13),(Q26),124,0.961240,1,"(Q13, Q26)",2
1050,(Q17),(Q26),132,0.956522,1,"(Q17, Q26)",2
1054,(Q17),(Q27),132,0.956522,1,"(Q27, Q17)",2
1096,(Q24),(Q27),130,0.948905,1,"(Q27, Q24)",2
1065,(Q21),(Q26),165,0.948276,1,"(Q21, Q26)",2


In [11]:
# Sort questions by number of apparition in consequents
questions = list(correctedScientific.columns)
for q in questions:
    rulesDataframe[q+"c"] = rulesDataframe["consequents"].apply(lambda x: 1 if q in x else 0)
occurenceInConsequents = rulesDataframe.loc[:,"Q1c":"Q27c"].sum(axis=0)
occurenceInConsequents.sort_values(inplace=True, ascending=False)
occurenceInConsequents

Q26c    415
Q19c    337
Q27c    307
Q21c    281
Q20c    214
Q1c     170
Q15c     25
Q17c     10
Q22c      0
Q9c       0
Q2c       0
Q3c       0
Q4c       0
Q5c       0
Q6c       0
Q7c       0
Q8c       0
Q11c      0
Q10c      0
Q23c      0
Q12c      0
Q13c      0
Q25c      0
Q16c      0
Q18c      0
Q24c      0
Q14c      0
dtype: int64

In [12]:
# Sort questions by number of apparition in antecedants
for q in questions:
    rulesDataframe[q+"a"] = rulesDataframe["antecedants"].apply(lambda x: 1 if q in x else 0)
occurenceInAntecedants = rulesDataframe.loc[:,"Q1a":"Q27a"].sum(axis=0)
occurenceInAntecedants.sort_values(inplace=True, ascending=False)
occurenceInAntecedants

Q19a    389
Q21a    384
Q26a    348
Q20a    319
Q27a    299
Q15a    258
Q1a     214
Q22a    184
Q2a      93
Q17a     88
Q25a     71
Q12a     27
Q14a      9
Q4a       7
Q24a      5
Q5a       4
Q11a      3
Q3a       2
Q13a      1
Q23a      1
Q16a      0
Q9a       0
Q8a       0
Q7a       0
Q6a       0
Q18a      0
Q10a      0
dtype: int64